In [14]:
def music_recomm():
    try:
        import pandas as pd
        import numpy as np
        from sklearn.preprocessing import MinMaxScaler
        from sklearn.metrics.pairwise import cosine_similarity
        from sklearn.cluster import KMeans
        import warnings
        warnings.filterwarnings("ignore")
        
        df = pd.read_csv(r"C:\Users\maner\OneDrive\Desktop\New folder\Pandas dataset\proj datasets\music\tracks_features.csv\tracks_features.csv")
        df.release_date = pd.to_datetime(df.release_date, errors = 'coerce')
        df = df.dropna()
        df['quarter'] = df.release_date.dt.quarter
        df.explicit = df.explicit.astype('int')
        train_features = ['explicit', 'danceability', 'energy','key', 'loudness', 'mode', 'speechiness', 'acousticness',
                      'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','time_signature']
        training_df = df[train_features]
        scale = MinMaxScaler()
        scale.fit(training_df)
        training_df = pd.DataFrame(scale.transform(training_df), 
                                    index=df.id,
                                    columns=training_df.columns)
        training_df = training_df[training_df.columns].astype('float32')
        kmeans = KMeans(n_clusters=10)
        kmeans.fit(training_df)
        training_df['cluster'] = kmeans.labels_
        df['cluster'] = kmeans.labels_

        ip = input("Enter the Song Name --> ")
        if df[df.name.str.lower() == ip.lower()].shape[0] > 1:
            print(f"Many matches found for song {ip}")
            album = input("Please Enter the album name --> ")
            idx = df[(df.name.str.lower() == ip.lower())&(df.album.str.lower() == al.lower())]['id'][0]
            clstr = df[(df.name.str.lower() == ip.lower())&(df.album.str.lower() == al.lower())]['cluster'][0]
        else :
            idx = list(df[df.name.str.lower() == ip.lower()]['id'])[0]
            clstr = list(df[df.name.str.lower() == ip.lower()]['cluster'])[0]
        i = int(input("How many simillar songs are needed --> "))
        q = input("Do you need recommendation of specific period (y/n) --> ")
        if q.lower() == 'n':
            recomm_playlist = df[(df.cluster == clstr) & (df.id != idx)][['id','name','album','artists']].sample(i)
            return recomm_playlist
        else:
            year = int(input("enter the year --> "))
            qtr = int(input("enter the quarter (1/2/3/4) --> "))
            df1 = df[(df.year == year) & (df.quarter == qtr)& (df.cluster == clstr)]
            df1 = pd.concat([df1,df[df.id == idx]])
            training_df1 = df1[train_features]
            scale1 = MinMaxScaler()
            training_df1 = pd.DataFrame(scale.transform(training_df1), 
                                    index=df1.id,
                                    columns=training_df1.columns)
            cos_data = cosine_similarity(training_df1)
            cos_df = pd.DataFrame(cos_data, index = list(training_df1.index), columns = list(training_df1.index))
            recom_ids = list(cos_df.iloc[::,-1][1:i+1].index)
            recom1 = pd.DataFrame()
            for ids in recom_ids:
                recom1 = pd.concat([recom1,df1[df1.id == ids][['id','name','album','artists']]])
            return recom1
    except:
        print(f"sorry! input song is not in our database")

In [8]:
ip = 'Testify'
al = 'The Battle Of Los Angeles'

In [9]:
song = 'If The Heavens Should Fall'

In [16]:
mus = music_recomm()

Enter the Song Name --> If The Heavens Should Fall
How many simillar songs are needed --> 5
Do you need recommendation of specific period (y/n) --> n


In [17]:
mus

,id,name,album,artists
1096027,23MWEPRoxPRJO78UYY9wfZ,Hameh Abi,Dar Mah-e-Baran (In The Season of Rain),"['Mohammad Nouri', 'Shahram Golparian']"
851622,16zyjYvaQhMODaHKfGnsBZ,Plug It In & Turn Me On,Plug It In,['Krezip']
663997,7IheDpPevBzlJaHFLoOFJA,Standing Strong - Bonus Track,Ironbark,['The Waifs']
718732,6snN8NUjM41uWD1yNn8O3t,I Want You,Notorious,['Adelitas Way']
320231,4vuH5hYzssnaq6SyRaCNZA,Boogie Rambler,The Alligator Records 30th Anniversary Collection,"['Lonnie Brooks', 'Long John Hunter', 'Phillip..."
